# Getting and filtering data from production redis

In [1]:
from urllib.parse import urlparse
from dotenv import load_dotenv
import os
from pathlib import Path
import json
import redis
import pandas as pd # only if needed for filtering of data

In [2]:
# load .env file
env_path = Path('../') / '.env'
load_dotenv(env_path)

True

In [3]:
def get_cic_stats_from_redis(redis_url):
    # Connect to Redis database
    parsed_url = urlparse(redis_url)
    r = redis.Redis(host=parsed_url.hostname,
                    port=parsed_url.port,
                    db=0,
                    password=parsed_url.password,
                    username=parsed_url.username)
    
    # get objects from redis
    redis_objects = r.mget(r.keys(pattern="cic:*CIC*lastStat*"))
    results = []
    for obj in redis_objects:
        try:
            results.append(json.loads(obj.decode()))
        except:
            pass
    return results

### Example of usage: getting all cic's connected via Wifi

In [ ]:
# get cic data from redis
REDIS_URL = os.getenv("REDISPROD")

redis_data = get_cic_stats_from_redis(REDIS_URL)

df_redis = pd.json_normalize(redis_data)

# select list of cic ids which are connected via Lte
cic_ids = df_redis[(df_redis['system.quattBuild'] != "2.0.1")][['system.quattId', 'system.quattBuild','time.tsHuman', 'system.isLteConnected','system.isWifiReachable','system.isEthernetReachable']]

# print list of cic ids
print(cic_ids)

      hp2        time.ts              time.tsHuman  hp1.modbusSlaveId  \
6     NaN  1686147937826  2023-06-07T14:25:37.827Z                  1   
7     NaN  1684389518209  2023-05-18T05:58:38.209Z                  1   
16    NaN  1686147946029  2023-06-07T14:25:46.029Z                  1   
19    NaN  1686147932448  2023-06-07T14:25:32.449Z                  1   
25    NaN  1683789591391  2023-05-11T07:19:51.391Z                  1   
...   ...            ...                       ...                ...   
1452  NaN  1683789598001  2023-05-11T07:19:58.002Z                  1   
1454  NaN  1686145474555  2023-06-07T13:44:34.555Z                  1   
1459  NaN  1686147942366  2023-06-07T14:25:42.366Z                  1   
1463  NaN  1684961999959  2023-05-24T20:59:59.959Z                  1   
1482  NaN  1682937361308  2023-05-01T10:36:01.308Z                  1   

      hp1.setMainWorkingMode  hp1.setCompressorFrequencyLevel  \
6                        0.0                              

In [7]:
cic_ids

,hp2,time.ts,time.tsHuman,hp1.modbusSlaveId,hp1.setMainWorkingMode,hp1.setCompressorFrequencyLevel,hp1.setCirculatingPumpRelay,hp1.setCirculatingPumpDutyCycle,hp1.waterPumpLevel,hp1.getMainWorkingMode,...,system.externalEnergyMeter1Flow,system.externalEnergyMeter1Supply,system.externalEnergyMeter1Return,system.externalEnergyMeter1EnergyCooling,system.externalEnergyMeter2Energy,system.externalEnergyMeter2Power,system.externalEnergyMeter2Flow,system.externalEnergyMeter2Supply,system.externalEnergyMeter2Return,system.externalEnergyMeter2EnergyCooling
6,NaN,1686147937826,2023-06-07T14:25:37.827Z,1,0.0,0.0,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,1684389518209,2023-05-18T05:58:38.209Z,1,0.0,0.0,False,0.0,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,1686147946029,2023-06-07T14:25:46.029Z,1,0.0,0.0,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,1686147932448,2023-06-07T14:25:32.449Z,1,0.0,0.0,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,1683789591391,2023-05-11T07:19:51.391Z,1,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,NaN,1683789598001,2023-05-11T07:19:58.002Z,1,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1454,NaN,1686145474555,2023-06-07T13:44:34.555Z,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,NaN,1686147942366,2023-06-07T14:25:42.366Z,1,0.0,0.0,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1463,NaN,1684961999959,2023-05-24T20:59:59.959Z,1,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
cic_ids[['system.quattId', 'system.quattBuild','time.tsHuman', 'system.isLteConnected','system.isWifiReachable','system.isEthernetReachable']]